# Introduction to SQLAlchemy: Schema & Metadata

In [1]:
# The structure of a relational schema is represented in Python
# using MetaData, Table, and other objects.

from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String

metadata = MetaData()


# resemble a create table statement in SQL
# ITs desig to look like the same way you do DDL(data definition language)
user_table = Table('user', metadata,
               Column('id', Integer, primary_key=True),
               Column('name', String),
               Column('fullname', String)
             )

## Table provides a single point of information regarding
## The structure of a table in a schema

In [3]:
user_table.name

'user'

## The `.c` attribute of Table is an associative array
## of column obhects, keyd on name

In [4]:
user_table.c.name

Column('name', String(), table=<user>)

## It's a bit like a python dictionary but not really

In [5]:
print(user_table.c)

['user.id', 'user.name', 'user.fullname']


In [7]:
user_table.columns.keys()

['id', 'name', 'fullname']

## Column itself has information about each column, such as name & type

In [8]:
user_table.c.name.name

'name'

In [10]:
user_table.c.name.type

String()

## Table has other information availible, such as the collection of columns which comprise the table's primary key

In [11]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user>, primary_key=True, nullable=False))

## The table object is at the core of the SQL expression sytem -this is a quick preview of that

In [12]:
print(user_table.select())

SELECT "user".id, "user".name, "user".fullname 
FROM "user"


## Table and MEtadata objects cam be used to generate a schema in a database

In [13]:
from sqlalchemy import create_engine
engine = create_engine("sqlite://")  # in memory database
metadata.create_all(engine)  # create all the tabgles that dont exist

## Types are represented using objects such as String, Integer, Datetime.
## These objects can be specified as "class keywords", or can be instantiated with arguments

In [14]:
from sqlalchemy import String, Numeric, DateTime, Enum

fancy_table = Table('fancy', metadata,
                    Column('key', String(50), primary_key=True),
                    Column('timestamp', DateTime),
                    Column('amount', Numeric(10, 2)),
                    Column('type', Enum('a', 'b', 'c'))
                )

fancy_table.create(engine)

## Table metadata also allows  form constraints and index
## Foreignkey is used to link one column to a remote primary key

In [15]:
from sqlalchemy import ForeignKey
addresses_table = Table('address', metadata,
                    Column('id', Integer, primary_key=True),
                    Column('email_address', String(100), nullable=False),
                    Column('user_id', Integer, ForeignKey('user.id'))
                  )

addresses_table.create(engine)

# ForeignKey is a shortcut for ForeignKeyConstraint,
# which should be used for composite references.

In [16]:
from sqlalchemy import Unicode, UnicodeText, DateTime
from sqlalchemy import ForeignKeyConstraint

story_table = Table('story', metadata,
               Column('story_id', Integer, primary_key=True),
               Column('version_id', Integer, primary_key=True),
               Column('headline', Unicode(100), nullable=False),
               Column('body', UnicodeText)
          )

published_table = Table('published', metadata,
            Column('pub_id', Integer, primary_key=True),
            Column('pub_timestamp', DateTime, nullable=False),
            Column('story_id', Integer),
            Column('version_id', Integer),
            ForeignKeyConstraint(
                            ['story_id', 'version_id'],
                            ['story.story_id', 'story.version_id'])
                )

# create_all() by default checks for tables existing already

In [17]:
metadata.create_all(engine)

# title:: Exercises
1. Write a Table construct corresponding to this CREATE TABLE
   statement.
```sql
CREATE TABLE network (
     network_id INTEGER PRIMARY KEY,
     name VARCHAR(100) NOT NULL,
     created_at DATETIME NOT NULL,
     owner_id INTEGER,
     FOREIGN KEY owner_id REFERENCES user(id)
)
```
2. Then emit metadata.create_all(), which will emit CREATE TABLE for this table (it will skip those that already exist).

The necessary types are imported here:

In [20]:
from sqlalchemy import Integer, String, DateTime

network_table = Table('network', metadata,
                     Column('network_id', Integer, primary_key=True,),
                     Column('name', String, nullable=False,),
                     Column('created_at', DateTime,nullable=False),
                     Column('owner_id', Integer, ForeignKey('user.id')),
                     )

InvalidRequestError: Table 'network' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [21]:
print(network_table.c)

['network.network_id', 'network.name', 'network.created_at', 'network.owner_id']


In [24]:
print(network_table.c.keys())

['network_id', 'name', 'created_at', 'owner_id']



# Reflection
# 'reflection' refers to loading Table objects based on
# reading from an existing database.

In [25]:
metadata2 = MetaData()

user_reflected = Table('user', metadata2, autoload=True, autoload_with=engine)

In [26]:
### slide:: i
print(user_reflected.c)

['user.id', 'user.name', 'user.fullname']



# Information about a database at a more specific level is available
# using the Inspector object.

In [27]:
from sqlalchemy import inspect

inspector = inspect(engine)

# the inspector provides things like table names:

In [28]:
inspector.get_table_names()

['address', 'fancy', 'published', 'story', 'user']


# column information

In [30]:
from pprint import pprint

In [36]:
#pprint(inspector.get_columns('address'))
inspector.get_columns('address')

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'email_address',
  'type': VARCHAR(length=100),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'user_id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]


# constraints

In [37]:
inspector.get_foreign_keys('address')

[{'name': None,
  'constrained_columns': ['user_id'],
  'referred_schema': None,
  'referred_table': 'user',
  'referred_columns': ['id'],
  'options': {}}]

In [ ]:
### slide::
### title:: Exercises
#
# 1. Using 'metadata2', reflect the "network" table in the same way
#    we just did 'user', then display the columns (or bonus, display
#    just the column names)
#
# 2. Using "inspector", print a list of all table names that
#    include a column called "story_id"
#